# Optimizaciones de objetivos

Este cuaderno demuestra un caso de uso de la optimización de objetivos de Amazon Personalize. A menudo, hay otros factores que influyen en las recomendaciones y la optimización de objetivos puede utilizarse para proporcionar datos adicionales que impulsen el modelo.

Este ejemplo supone que estamos proporcionando recomendaciones para un servicio de suscripción de video bajo demanda (VOD). El servicio permite a los usuarios un acceso ilimitado al catálogo de videos, aunque los títulos disponibles en su catálogo tienen diferentes condiciones de licencia en función del acuerdo con el propietario del contenido. Por medio de las recomendaciones que dan peso a los contenidos con menores costos de licencia, el servicio de streaming puede reducir los costos de las licencias sin dejar de satisfacer a los clientes.

Una vez más, los datos provienen del proyecto [MovieLens](https://movielens.org/). Puede aprender más sobre los datos y los usos potenciales haciendo una búsqueda en la web durante cualquiera de los períodos de espera en las celdas a continuación. Complementará los datos de MovieLens con un campo adicional de regalías y generará los datos que proporcionarán los costos de regalías al conjunto de datos. 

## Cómo utilizar el cuaderno

El código se divide en celdas como la siguiente. En la parte superior de esta página, se encuentra un botón triangular `Run` (Ejecutar) en el que puede hacer clic para ejecutar cada celda y pasar a la siguiente, o puede presionar `Shift` + `Enter`mientras está en la celda para ejecutarla y pasar a la siguiente.

A medida que una celda se ejecuta, verá una línea al costado que muestra un `*` mientras la celda se está ejecutando o se actualizará a un número para indicar la última celda que terminó de ejecutarse luego de que haya terminado de ejecutar todo el código dentro de una celda.


Solo debe seguir las instrucciones que aparecen a continuación y ejecutar las celdas para comenzar a utilizar la optimización de objetivos de Amazon Personalize.

## Importaciones 

Python viene con una amplia colección de bibliotecas y necesitamos importarlas, así como las que se instalan para ayudarnos, como [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK para Python) y [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), que son herramientas básicas de la ciencia de datos.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError

!conda install -y -c conda-forge unzip

pd.options.mode.chained_assignment = None

A continuación, querrá validar que su entorno puede comunicarse correctamente con Amazon Personalize. Las líneas que figuran a continuación hacen precisamente eso.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configurar los datos

Los datos se importan a Amazon Personalize a través de Amazon S3. A continuación, especificaremos un nombre de archivo que se utiliza para escribir los archivos localmente antes de cargarlos a S3.


In [ ]:
filename = "movie-lens-100k.csv"
items_filename = "movie-lens-items.csv"

### Descargar, preparar y cargar los datos de entrenamiento

En la actualidad, aún no tiene los datos de MovieLens cargados de forma local para examinarlos. Ejecute las siguientes líneas para descargar la última copia y examinarla rápidamente.

#### Descargar y explorar el conjunto de datos

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### Preparar y cargar los datos

El siguiente código carga la información de la película desde la descarga de MovieLens. Proporciona información sobre el título, la fecha de estreno, el enlace a IMDB y una lista de columnas para capturar los géneros que se aplican al título.

No hay un campo de regalías, ya que el taller está generando una regalía ficticia y lo hará en el siguiente paso.








In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|',encoding='latin-1', names=['ITEM_ID', 'TITLE', 'RELEASE_DATE', 'VIDEO_RELEASE_DATE', 'IMDB_URL', 'MISC', 'ACTION_GENRE', 'ADVENTURE_GENRE', 'ANIMATION_GENRE', 'CHILDRENS_GENRE','COMEDY_GENRE', 'CRIME_GENRE', 'DOCUMENTARY_GENRE','DRAMA_GENRE','FANTASY_GENRE', 'FILMNOIR_GENRE','HORROR_GENRE', 'MUSICAL_GENRE','MYSTERY_GENRE', 'ROMANCE_GENRE', 'SCIFI_GENRE', 'THRILLER_GENRE', 'WAR_GENRE', 'WESTERN_GENRE'     ])
items

#### Agregar los datos de regalías y géneros
Estamos asignando un valor al campo REGALÍA con una distribución uniforme de los valores 0,0, 0,005, 0,01, 0,015, 0,02, 0,025, 0,05, 0,10. Esto sitúa a la mayoría de las películas con una regalía relativamente baja o nula y a un pequeño número de películas con una regalía más alta. 

Observe en el gráfico de barras de abajo la distribución uniforme de los títulos con cada valor de regalía.

El resultado del campo GÉNERO de MovieLens anterior es categórico y cada columna indica si el título pertenece al género, pero los elementos cargados en la personalización pueden aceptar matrices. Algunos códigos de panda para reunir los géneros en una columna delimitada por tuberías que enumera todos los géneros.



In [ ]:
pd.set_option('display.max_rows', 10)

royaltyvalues = [0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.10]
items.loc[:,'ROYALTY'] = items['ITEM_ID'].map(lambda x: royaltyvalues[x%8])

items.loc[:,'GENRE']='' 

for col_name in items.columns:
    if col_name.endswith('_GENRE'):
        items.loc[items[col_name]==1,'GENRE']= items['GENRE']+'|'+ col_name[:-6]

items = items[['ITEM_ID', 'TITLE','ROYALTY', 'GENRE']]
items.loc[:,'GENRE'] = items['GENRE'].str[1:]


items.loc[:,'ROYALTY'].value_counts().plot.bar()
items.head(10)


#### Ajuste el valor de la regalía a un número negativo.

La optimización del objetivo optimizará los valores más altos de un campo. En este caso, queremos equilibrar la relevancia de la película frente a los gastos de regalías en los que incurrirá el servicio de streaming.

Para impulsar las películas con las regalías más bajas, las películas con las regalías más bajas necesitan los valores numéricos más altos. En este caso, convertiremos las regalías en un número negativo. Esto se hace multiplicando -1 por el valor absoluto de la regalía.

Nota: El valor absoluto se utiliza para que esta celda pueda ejecutarse dos veces con los mismos resultados, lo cual es útil en un entorno de taller.

In [ ]:
items.loc[:,'ROYALTY'] = -1 * abs(items['ROYALTY'])
items

## Configurar un bucket de S3 y un rol de IAM <a class="anchor" id="bucket_role"></a>
Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

Utilice los metadatos almacenados en la instancia subyacente a este cuaderno de Amazon SageMaker para determinar la región en la que opera. Si utiliza un cuaderno de Jupyter fuera de Amazon SageMaker, simplemente defina la región como una cadena a continuación. El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    notebook_data = json.load(notebook_info)
    resource_arn = notebook_data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Los nombres de los buckets de Amazon S3 son exclusivos a nivel global. Para crear un nombre de bucket exclusivo, el siguiente código agregará la cadena `personalize-objective-optimization-` a su número de cuenta de AWS. Luego, crea un bucket con este nombre en la región descubierta en la celda anterior.

In [ ]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-objective-optimization-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### Cargar los datos a S3

Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de los elementos. 

In [ ]:
items = items[['ITEM_ID', 'TITLE', 'GENRE', 'ROYALTY']]
items_dataset = items[['ITEM_ID','ROYALTY', 'GENRE']]

items_dataset.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

### Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

#### Crear el esquema de elementos del título de la película
Necesitaremos dos esquemas, uno para los títulos de las películas, que será del tipo elementos, y un segundo que definirá la estructura de las interacciones.

In [ ]:
schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "ROYALTY",
            "type": "float"
        },        {
            "name": "GENRE",
            "type": [
                "null",
                "string"
              ],
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_item_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-item-schema"+suffix,
    schema = json.dumps(schema)
)

item_schema_arn = create_item_schema_response['schemaArn']
print(json.dumps(create_item_schema_response, indent=2))

A partir de aquí, seguiremos el mismo proceso de la [primera campaña](https://github.com/aws-samples/amazon-personalize-samples/blob/master/getting_started/notebooks/1.Building_Your_First_Campaign.ipynb), cargando las interacciones. Solo queremos incluir interacciones con al menos una calificación de tres y no queremos recomendar películas que no gusten a los espectadores.

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

### Crear un esquema de interacciones

Un componente central de cómo Personalize entiende sus datos proviene del esquema que se define a continuación. Esta configuración indica al servicio cómo digerir los datos proporcionados a través de su archivo CSV. Observe que las columnas y los tipos se alinean con lo que había en el archivo que creó anteriormente.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-schema"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### Crear y esperar el grupo de conjuntos de datos

La agrupación más grande en Personalize es un grupo de conjuntos de datos, esto aislará sus datos, rastreadores de eventos, soluciones y campañas. Agrupe elementos que comparten una colección de datos común. Siéntase libre de modificar el nombre que aparece a continuación si lo desea.

#### Crear un grupo de conjuntos de datos

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-objective-optmization-demo-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Esperar a que el grupo de conjunto de datos tenga estado ACTIVO

Debe estar activo antes de que podamos utilizar el grupo de conjunto de datos en cualquiera de los siguientes artículos. Ejecute la celda a continuación y espere a que se muestre activo.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Crear conjuntos de datos

Después del grupo, el próximo paso es crear los conjuntos de datos reales, en este ejemplo crearemos 1 para los datos de las interacciones y otro para los datos de los elementos. Ejecute las celdas a continuación para crearlo.

In [ ]:
def create_dataset(dataset_type, schema_arn, name):
    create_dataset_response = personalize.create_dataset(
        name = name,
        datasetType = dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = schema_arn
    )
    dataset_arn = create_dataset_response['datasetArn']

    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = dataset_arn
        )
        status = describe_dataset_response["dataset"]["status"]
        print("Dataset: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(10)
    return dataset_arn


In [ ]:
interaction_dataset_arn = create_dataset("INTERACTIONS", schema_arn, 'personalize-objective-optmization-interactions-'+suffix)


print('interaction_dataset_arn: ' + interaction_dataset_arn)


In [ ]:
item_dataset_arn = create_dataset("ITEMS", item_schema_arn, 'personalize-objective-optmization-items-'+suffix)
print('item_dataset_arn: ' + item_dataset_arn)

#### Crear un rol de Personalize

Además, Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Las líneas que aparecen a continuación otorgan eso.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importar los datos

Anteriormente, creó el grupo de conjunto de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos de S3 en Amazon Personalize para su uso, a medida que crea su modelo. Cargaremos múltiples importaciones, por lo que se llamará a la función que aparece a continuación para iniciar el trabajo de importación y, a continuación, supervisar la finalización del trabajo de importación.

#### Crear un trabajo de importación de conjuntos de datos

In [ ]:
def create_dataset_import_job(dataset_arn, dataLocation, name):
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = name,
        datasetArn = dataset_arn,
        dataSource = {
            "dataLocation": dataLocation
        },
        roleArn = role_arn
    )

    dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dataset_import_job_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        print("DatasetImportJob: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)
    return dataset_import_job_arn

#### Cargar las interacciones
El siguiente trabajo de importación cargará el conjunto de datos de interacción.

In [ ]:
dataset_import_job_arn = create_dataset_import_job(interaction_dataset_arn, "s3://{}/{}".format(bucket_name, filename), "personalize-objective-optimization-interaction-"+suffix)
print('dataset_import_job_arn: ' + dataset_import_job_arn)

#### Cargar los títulos de las películas en el conjunto de datos de elementos

Puede pasar un tiempo antes de que el trabajo de importación se complete. Espere hasta que vea que está activo a continuación.

In [ ]:
item_dataset_import_job_arn = create_dataset_import_job(item_dataset_arn, "s3://{}/{}".format(bucket_name, items_filename), "personalize-objective-optimization-item-"+suffix)
print('item_dataset_import_job_arn: ' + item_dataset_import_job_arn)

## Crear la solución y la versión

En Amazon Personalize, un modelo entrenado se denomina solución, cada solución puede tener muchas versiones específicas relacionadas con un volumen determinado de datos cuando se entrenó el modelo.

Para comenzar, enumeraremos todas las recetas que se admiten. Una receta es un algoritmo que aún no ha sido entrenado en sus datos. Después del listado, seleccionará uno y lo utilizará para construir su modelo.

### Seleccionar receta

#### Personalización del usuario
La receta [User-Personalization](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) (aws-user-personalization) está optimizada para todos los escenarios de recomendación USER_PERSONALIZATION. Utiliza la exploración automática de elementos para hacer recomendaciones.

Con la exploración automática, Amazon Personalize prueba automáticamente diferentes recomendaciones de elementos, aprende de la forma en que los usuarios interactúan con estos elementos recomendados y potencia las recomendaciones de elementos que impulsan un mejor compromiso y conversión. Esto mejora el descubrimiento de elementos y la participación cuando tiene un catálogo que cambia rápidamente, o cuando los nuevos elementos, como las noticias o las promociones, son más relevantes para los usuarios cuando están frescos.

Puede equilibrar cuánto explorar (donde los elementos con menos datos de interacciones o relevancia se recomiendan con más frecuencia) frente a cuánto explotar (donde las recomendaciones se basan en lo que conocemos o en la relevancia). Amazon Personalize ajusta automáticamente las futuras recomendaciones en función de los comentarios implícitos de los usuarios.

En primer lugar, seleccione la receta buscando el ARN en la lista de recetas anterior.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### Crear y esperar una solución

Primero creará la solución con la API, luego creará una versión. Tardará varios minutos en entrenar el modelo y crear así su versión de una solución. Una vez que se pone en marcha y se ven las notificaciones en curso es un buen momento para tomar un descanso, tomar un café, etc.

La función acepta el ajuste de sensibilidad del objetivo, que puede ser APAGADO, BAJO, MEDIO o ALTO. Esto ajustará la ponderación que impulsa el impacto del objetivo en el modelo.

La función crea la solución y la versión de la solución inicial para esa solución.

#### Crear una solución

In [ ]:
def create_solution(name, objectiveSensitivity):
    create_solution_response = personalize.create_solution(
        name = name,
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn,
        solutionConfig = {
            "optimizationObjective": {
                "itemAttribute": "ROYALTY",
                "objectiveSensitivity":objectiveSensitivity
            }
        }
    )
    
    solution_arn = create_solution_response['solutionArn']
    
    print('solutionArn:' + solution_arn)

    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn
    )

    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print('solution_version_arn: ' + solution_version_arn)

    return {
        "solution_arn": solution_arn,
        "solution_version_arn": solution_version_arn
    }
    
def waitForSolutionVersion(solution_version_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response["solutionVersion"]["status"]
        print("SolutionVersion: {} {}".format(solution_version_arn, status))

        if status == "ACTIVE" or status == "CREATE FAILED"  or status == "CREATE STOPPING":
            break

        time.sleep(60)

#### Crear versiones de la solución
Cree 3 soluciones diferentes, una con la optimización de objetivos desactivada, otra con un ajuste BAJO y otra con un ajuste ALTO. Esto pondrá en marcha 3 soluciones en paralelo, y el siguiente paso esperará a que se completen.

In [ ]:
high_solution = create_solution('movie-recommendation-low-royalties-'+suffix, 'HIGH')
low_solution = create_solution('movie-recommendation-medium-royalties-'+suffix, 'LOW')
no_objective_optimization_solution = create_solution('movie-recommendation-max-relevance-'+suffix, 'OFF')

#### Esperar a que las versiones de la solución tenga un estado ACTIVO.

Esto tardará entre 40y 50 minutos aproximadamente.

In [ ]:
waitForSolutionVersion(low_solution['solution_version_arn'])
waitForSolutionVersion(no_objective_optimization_solution['solution_version_arn'])
waitForSolutionVersion(high_solution['solution_version_arn'])

#### Obtener las métricas de las versiones de la solución

Una vez que disponga de su solución y de su versión, puede obtener las métricas para juzgar su rendimiento. Estas métricas no son especialmente buenas, ya que se trata de un conjunto de datos de demostración, pero con conjuntos de datos más grandes y complejos se deberían ver mejoras.

Puede ver las diferencias en la calidad del modelo en función del impacto de las optimizaciones de los objetivos.

In [ ]:
def get_solution_metrics(solutions):
    metricdata = { "name": []}
    
    for key in solutions:
        solution = solutions[key]
        
        metricdata["name"].append(key)
        
        get_solution_metrics_response = personalize.get_solution_metrics(
            solutionVersionArn = solution['solution_version_arn']
        )

        for metricname in get_solution_metrics_response['metrics']:
            if not metricname in metricdata:
                metricdata[metricname] = []
                
            metricdata[metricname].append( get_solution_metrics_response['metrics'][metricname])
            
        # print(json.dumps(get_solution_metrics_response, indent=2))
    return pd.DataFrame.from_dict(metricdata);

metrics = get_solution_metrics({
    "no-optimization": no_objective_optimization_solution,
    "low-optimization": low_solution,
    "high-optimization": high_solution,
})

metrics

Recomendamos leer [la documentación](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) para comprender las métricas, pero, además, copiamos partes de la documentación a continuación para mayor comodidad.

Es necesario que comprenda los siguientes términos relativos a la evaluación en Personalize:

- *Recomendación relevante* se refiere a una recomendación que coincide con un valor en los datos de prueba para el usuario en particular.
- *Rango* se refiere a la posición de un elemento recomendado en la lista de recomendaciones. Se supone que la posición 1 (la primera de la lista) es la más relevante para el usuario.
- *Consulta* se refiere al equivalente interno de una llamada a GetRecommendations.

Las métricas producidas por Personalize son las siguientes:

- coverage: La proporción de elementos únicos recomendados de todas las consultas sobre el número total de elementos únicos en los datos de entrenamiento (incluye tanto los conjuntos de datos de elementos como de interacciones).
- mean_reciprocal_rank_at_25: La [media de los rangos recíprocos](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la primera recomendación relevante entre las 25 primeras recomendaciones sobre todas las consultas. Esta métrica es apropiada si está interesado en la recomendación de mayor rango.
- normalized_discounted_cumulative_gain_at_K: La ganancia descontada supone que las recomendaciones más bajas de una lista de recomendaciones son menos relevantes que las más altas. Por lo tanto, cada recomendación se descuenta (se le da un peso menor) por un factor que depende de su posición. A fin de obtener la [ganancia descontada acumulada](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) en K, se suma cada recomendación descontada relevante en las  recomendaciones más importantes de K. La ganancia acumulada descontada normalizada (NDCG) es la DCG dividida por la DCG ideal, de manera que la NDCG está entre 0 y 1. (El DCG ideal es aquel en el que las recomendaciones más importantes de K están ordenadas por relevancia). Amazon Personalize utiliza un factor de ponderación de 1/log (1 + posición), donde la parte superior de la lista es la posición 1. Esta métrica recompensa los elementos relevantes que aparecen cerca de la parte superior de la lista, porque la parte superior de una lista suele llamar más la atención.
- precision_at_K: El número de recomendaciones relevantes de las principales recomendaciones de K dividido por K. Esta métrica recompensa la recomendación precisa de los elementos relevantes.

## Crear y esperar una campaña

Ahora que tiene una versión de la solución que funciona, tendrá que crear una campaña para utilizarla con sus aplicaciones. Una campaña es una versión de la solución alojada; un punto de conexión que puede consultar para obtener recomendaciones. El precio se fija estimando la capacidad de rendimiento (solicitudes de personalización de los usuarios por segundo). Cuando implementa una campaña, se establece un valor mínimo de transacciones por segundo (TPS) (`minProvisionedTPS`). Este servicio, como muchos dentro de AWS, se escalará automáticamente en función de la demanda, pero si la latencia es crítica, es posible que desee aprovisionar por adelantado para una mayor demanda. Para esta demostración, todos los límites mínimos de rendimiento se establecen en 1. Para obtener más información, consulte la página de [precios](https://aws.amazon.com/personalize/pricing/).

Como se ha mencionado anteriormente, la receta de personalización del usuario utilizada para nuestra solución admite la exploración automática de elementos "fríos". Puede controlar el grado de exploración que se realiza cuando crea su campaña. El tipo de datos `itemExplorationConfig` admite parámetros `explorationWeight` y `explorationItemAgeCutOff`. El peso de la exploración determina la frecuencia con la que las recomendaciones incluyen elementos con menos datos de interacción o relevancia. Cuanto más se acerque el valor a 1,0, mayor será la exploración. A cero, no se produce ninguna exploración y las recomendaciones se basan en los datos actuales (relevancia). El límite de antigüedad de los elementos de exploración determina los elementos que deben explorarse en función del tiempo transcurrido desde la última interacción. Proporcione la antigüedad máxima del elemento, en días desde la última interacción, para definir el alcance de la exploración del elemento. Cuanto mayor sea el valor, más elementos se tendrán en cuenta durante la exploración. Para nuestra campaña, especificaremos un peso de exploración de 0,5.

#### Crear una campaña

In [ ]:
def create_campaign(solution, name):
    create_campaign_response = personalize.create_campaign(
        name = "personalize-demo-" + name + '-' + suffix,
        solutionVersionArn = solution['solution_version_arn'],
        minProvisionedTPS = 1,
        campaignConfig = {
            "itemExplorationConfig": {
                "explorationWeight": "0.5"
            }
        }
    )

    campaign_arn = create_campaign_response['campaignArn']
    print('campaign_arn:' + campaign_arn)
    return campaign_arn

def waitForCampaign(solution):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = solution['campaign_arn']
        )
        status = describe_campaign_response["campaign"]["status"]
        print("Campaign: {} {}".format(solution['campaign_arn'], status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)

#### Crear 3 campañas
Cree una campaña para cada una de las optimizaciones de objetivos, pero mantenga todas las demás configuraciones iguales para demostrar el impacto de la optimización de objetivos.

In [ ]:
high_solution['campaign_arn'] = create_campaign(high_solution, 'high')
low_solution['campaign_arn'] = create_campaign(low_solution, 'low')
no_objective_optimization_solution['campaign_arn'] = create_campaign(no_objective_optimization_solution, 'max_relevance')


#### Esperar a que la campaña tenga un estado ACTIVO

Esto debería llevar unos 10 minutos.

In [ ]:
waitForCampaign(high_solution)
waitForCampaign(low_solution)
waitForCampaign(no_objective_optimization_solution)

## Obtener muestras de recomendaciones

Cuando la campaña esté activa, estará lista para recibir recomendaciones. Primero tenemos que seleccionar un usuario de la colección al azar. A continuación, crearemos algunas funciones de ayuda para obtener la información de la película que se mostrará para las recomendaciones en vez de mostrar únicamente los ID.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE'] + '(' + f'{-1*items.iloc[movie_id]["ROYALTY"]:.2f}'+ ')'

def get_movie_royalty(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return -1*items.iloc[movie_id]["ROYALTY"]

#### Llamar a GetRecommendations

Con el usuario que obtuvo anteriormente, las líneas siguientes le proporcionarán recomendaciones y le mostrarán la lista de películas recomendadas.


In [ ]:
def get_recommendations(solution):
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = solution['campaign_arn'],
        userId = str(user_id),
    )
    # Update DF rendering
    pd.set_option('display.max_rows', 30)

    item_list = get_recommendations_response['itemList']

    recommendation_list = []

    total_royalties = 0.0
    
    for item in item_list:
        title = get_movie_title(item['itemId'])
        total_royalties = total_royalties + get_movie_royalty(item['itemId'])
        recommendation_list.append(title)
        
    recommendation_list.append('TOTAL ROYALTIES: '+ f'{total_royalties:.2f}')
    return recommendation_list

#### Comparar las recomendaciones
Crear un conjunto de recomendaciones para el mismo usuario a fin de comparar el impacto de la optimización del objetivo.

Observe el impacto del valor de la regalía que aparece entre paréntesis después del título y el año. Las películas más valoradas y con un alto nivel de regalías con la optimización objetiva desactivada tienden a aparecer más abajo en la lista, si es que aparecen, cuando la optimización objetiva está activada. 

Tenga en cuenta, también, el total de regalías de todos los títulos de cada conjunto de recomendaciones.

In [ ]:
recommendations_df = pd.DataFrame(get_recommendations(no_objective_optimization_solution), columns = ['ObjectiveOff'])
recommendations_df['LowObjective'] = get_recommendations(low_solution)
recommendations_df['HighObjective'] = get_recommendations(high_solution)
recommendations_df

## Revisión

El cuaderno mostró un ejemplo en el que el motor de recomendación de películas tenía en cuenta las regalías necesarias para pagar un título determinado. Si incluimos esta consideración en el algoritmo de recomendación, el servicio de streaming puede ofrecer buenas recomendaciones a un usuario, pero también minimizar los derechos que hay que pagar a los creadores de contenidos.

Observe que en el gráfico anterior, con la optimización de objetivos desactivada, la tasa de regalías entre paréntesis se distribuye de forma bastante uniforme, como cabría esperar, ya que no se tiene en cuenta. Sin embargo, en la columna de objetivo bajo, los valores son más bajos y la suma de las regalías es menor para la configuración de optimización de objetivo más alto.

## Limpiar

Limpiar recursos

In [ ]:
def delete_campaign(campaign_arn):
    delete_campaign_result = personalize.delete_campaign(campaignArn=campaign_arn )
    

def wait_for_delete_campaign(campaign_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_campaign_response = personalize.describe_campaign(
                campaignArn = campaign_arn
            )
            status = describe_campaign_response["campaign"]["status"]
            print("campaign: {}".format(status))

        except ClientError as e:
            print(e)
            break

        time.sleep(10)
    print('campaign ' + campaign_arn + ' deleted')
    
def delete_solution(solution_arn):
    delete_solution_result = personalize.delete_solution(solutionArn=solution_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours

def wait_for_delete_solution(solution_arn):
    while time.time() < max_time:
        
        try:
            describe_solution_response = personalize.describe_solution(
                solutionArn = solution_arn
            )
            status = describe_solution_response["solution"]["status"]
            print("Solution: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('Solution ' + solution_arn + ' deleted')
    
def delete_dataset(dataset_arn):
    delete_dataset_result = personalize.delete_dataset(datasetArn=dataset_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_response = personalize.describe_dataset(
                datasetArn = dataset_arn
            )
            status = describe_dataset_response["dataset"]["status"]
            print("dataset: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset ' + dataset_arn + ' deleted')
    
def delete_schema(schema_arn):
    delete_schema_result = personalize.delete_schema(schemaArn=schema_arn )
    

    print('schema ' + schema_arn + ' deleted')
    
def delete_dataset_group(dataset_group_arn):
    delete_dataset_group_result = personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_group_response = personalize.describe_dataset_group(
                datasetGroupArn = dataset_group_arn
            )
            status = describe_dataset_group_response["datasetGroup"]["status"]
            print("dataset_group: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset_group ' + dataset_group_arn + ' deleted')
    
def delete_all(solutions):
    for solution in solutions:
        delete_campaign(solution['campaign_arn'])
        
    for solution in solutions:
        wait_for_delete_campaign(solution['campaign_arn'])
    for solution in solutions:
        delete_solution(solution['solution_arn'])
    for solution in solutions:
        wait_for_delete_solution(solution['solution_arn'])


    

In [ ]:
delete_all([no_objective_optimization_solution, low_solution, high_solution] )

In [ ]:
delete_dataset(item_dataset_arn)
delete_dataset(interaction_dataset_arn)

In [ ]:
delete_schema(item_schema_arn)
delete_schema(schema_arn)

In [ ]:
delete_dataset_group(dataset_group_arn)

In [ ]:
iam = boto3.client("iam")

iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess')
iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
time.sleep(10) # propogation time

iam.delete_role(RoleName=role_name)

In [ ]:
! aws s3 rm --recursive s3://$bucket_name

In [ ]:
s3.delete_bucket(Bucket=bucket_name)